In [80]:
import pandas as pd

In [81]:
#adjust datapath to reflect the location of data in your file system
datapath = "/Users/paulreginato/Google Drive/genome seq in situ/Analysis/191202_embryo_analysis/embryo_seq_data_tables/201125_embryo_data_table_Y_nonchromosomal_fixed2.csv"
source_data = pd.read_csv(datapath,index_col=False)

In [82]:
# Call parent of origin for each cluster based on the SNP assignments for reads in the cluster by tallying
# the number of SNPs from each parent in the cluster and making the call based on majority rules. If the cluster
# cannot be assigned to a parent because it does not contain SNPs or contains the same number of SNPs from each 
# parent, leave it unclassified. If the cluster is an X chromosome and there is only one cluster in the cell,
# call parent of origin as maternal. Add the column 'cluster_hap' to indicate the parent call, which will be
# 0 if maternal, 1 if paternal, and -1 if unclassified. Add the column 'single_snp' to indicate whether the 
# cluster had only one SNP in the cluster. Add columns 'num_hap0' and 'num_hap1', which will keep track of the
# number of SNPs from each parent in the cluster. Takes a data table
def call_raw_haps(data):
    
    new_data=data.copy()
    
    #create the new columns
    new_data['cluster_hap']=-1
    new_data['single_snp']=0
    new_data['num_hap0']=0
    new_data['num_hap1']=0

    
    #loop through all clusters in all cells
    cell_indices = new_data.cell_index.unique()
    
    for cell_index in cell_indices:
        cell = new_data.loc[new_data.cell_index==cell_index]
        
        chr_nums = cell.chr.unique()
            
        for chr_num in chr_nums:
            chrom = cell.loc[cell.chr==chr_num]
            clust_nums = chrom.cluster.unique()
            
            #If there is only one cluster and it is an X chromosome, assign 'cluster_hap' 
            # to 0 (maternal).
            if ((len(clust_nums)==1)&(chr_num==20)):
                new_data.loc[chrom.index,'cluster_hap']=0
                continue
            
            for clust_num in clust_nums:  
                clust = chrom.loc[chrom.cluster==clust_num]
                
                #get the number of SNPs for each hap in the cluster
                num_hap0=len(clust.loc[clust.hap_assignment==0])
                num_hap1=len(clust.loc[clust.hap_assignment==1])
                
                #record the number of SNPs in each hap
                new_data.loc[clust.index,'num_hap0']=num_hap0
                new_data.loc[clust.index,'num_hap1']=num_hap1
                
                #compare the number of SNPs in each hap and assign haplotype based on majority
                if num_hap0>num_hap1:
                    new_data.loc[clust.index,'cluster_hap']=0
                    win_num = num_hap0
                    lose_num = num_hap1
                elif num_hap1>num_hap0:
                    new_data.loc[clust.index,'cluster_hap']=1
                    win_num = num_hap1
                    lose_num = num_hap0
                    
                #fill in the column 'single_snp'
                if win_num==1:
                    new_data.loc[clust.index,'single_snp']=1
                    
    return new_data


# revise the raw haplotype calls for each chromosome based on the identities of the other clusters. Add
# the new column 'cluster_hap_imputed'. Rules:
#1. If the cluster number is -1, assign 'cluster_hap_imputed' to -1.
#2. If one cluster is assigned to parent of origin and the other is not, call the unclassified cluster as the 
# opposite of the one that is classified.
#3. If the parent of origin calls are the same but one of the clusters contained only one SNP and the other 
#had multiple, change the one with only one SNP.
#4. If the parent of origin calls are the same and Rule 3 can't be applied, assign 'cluster_hap_imputed' to -1.
#5. If the chromosome is aneuploid, assign 'cluster_hap_imputed' to the same value as the raw assignment.
#6. If there is only one cluster and it is an X chromosome, assign 'cluster_hap_imputed' to 0 (maternal).


#takes a dataframe outputted by call_raw_haps()

def revise_raw_hap_calls(data):
    num_disagreements = 0
    new_data=data.copy()
    new_data['cluster_hap_imputed']=-1
    
    #loop through every chromosome in every cell
    cell_indices = new_data.cell_index.unique()
    
    for cell_index in cell_indices:
        cell = new_data.loc[new_data.cell_index==cell_index]
        
        chr_nums = cell.chr.unique()
            
        for chr_num in chr_nums:
            chrom = cell.loc[cell.chr==chr_num]
            clust_nums = chrom.cluster.unique()
            
            #Rule 1: if any of the clusters have cluster number -1, assign 'cluster_hap_imputed' to -1 
            if -1 in list(chrom.cluster):
                continue
            
            #Rule 6: If there is only one cluster and it is an X chromosome, assign 'cluster_hap_imputed' 
            # to 0 (maternal).
            if ((len(clust_nums)==1)&(chr_num==20)):
                new_data.loc[chrom.index,'cluster_hap_imputed']=0
                continue
            
            #Rule 5: if there are more than two clusters, assign 'cluster_hap_imputed' to the same value 
            #as the raw assignment.
            if len(clust_nums)!=2:
                for clust_num in clust_nums:
                    clust = chrom.loc[chrom.cluster==clust_num]
                    clust_call = clust.iloc[0].cluster_hap
                    new_data.loc[clust.index,'cluster_hap_imputed']=clust_call
                continue
                    
            #If there are two clusters and neither are assigned cluster number -1, apply the other rules
            
            #Find the parent of origin assignments for each cluster
            clust1 = chrom.loc[chrom.cluster==1]
            clust2 = chrom.loc[chrom.cluster==2]
            c1_call = clust1.iloc[0].cluster_hap
            c2_call = clust2.iloc[0].cluster_hap
            
            #Rule 2: if the raw parent of origin calls are different leave them the same, unless one is 
            # unclassified, in which case update it to match the one that is classified. 
            if c1_call!=c2_call:
                if c1_call == -1:
                    new_data.loc[clust2.index,'cluster_hap_imputed']=c2_call
                    new_data.loc[clust1.index,'cluster_hap_imputed']=abs(c2_call-1)
                elif c2_call == -1:
                    new_data.loc[clust1.index,'cluster_hap_imputed']=c1_call
                    new_data.loc[clust2.index,'cluster_hap_imputed']=abs(c1_call-1)
                else:
                    new_data.loc[clust1.index,'cluster_hap_imputed']=c1_call
                    new_data.loc[clust2.index,'cluster_hap_imputed']=c2_call
                    
                continue
            
            #Rule 4: if the raw parent of origin calls are the same and both clusters had more than one SNP,
            # assign 'cluster_hap_imputed' to -1
            if ((clust1.iloc[0]['single_snp']==0)&(clust2.iloc[0]['single_snp']==0)):
                continue
            
            #Rule 3: if the parent of origin calls are the same but one of the clusters contained only one SNP 
            #and the other had multiple, change the one with only one SNP.
            if ((clust1.iloc[0]['single_snp']==1)&(clust2.iloc[0]['single_snp']!=1)):
                new_data.loc[clust2.index,'cluster_hap_imputed']=c2_call
                new_data.loc[clust1.index,'cluster_hap_imputed']=abs(c2_call-1)
                continue
            
            if ((clust2.iloc[0]['single_snp']==1)&(clust1.iloc[0]['single_snp']!=1)):
                new_data.loc[clust1.index,'cluster_hap_imputed']=c1_call
                new_data.loc[clust2.index,'cluster_hap_imputed']=abs(c1_call-1)
                continue
                
    return new_data

#Annotates the parent of origin assignments in the full dataset. Assigns outliers to the parent of origin
#of their associated cluster in the cluster_hap and cluster_hap_imputed columns. All read that are not part of a
#chromosome cluster are assigned -1 in those columns. Takes an unfiltered data table ('full_data') and annotates it
#with the parent of origin data from a filtered table, ie the output of revise_raw_hap_calls ('assigned_data').
def assign_haps_to_outliers(full_data,assigned_data):
    new_data = full_data.copy()
    new_data['cluster_hap']=-1
    new_data['cluster_hap_imputed']=-1
    
    #go through each cluster in each cell in the assigned_data. Map the parent of origin assignments from each
    #cluster in assigned_data to the same cluster in full_data, including outliers. This allows outliers in the
    #full_data to be assigned parent of origin, even though they were not used to decide the assignments.
    
    cell_indices = assigned_data.cell_index.unique()
    for ci in cell_indices:
        cell = assigned_data.loc[assigned_data.cell_index==ci]
        chr_nums = cell.chr.unique()
        for chr_num in chr_nums:
            chrom = cell.loc[cell.chr==chr_num]
            clust_nums = chrom.cluster.unique()
            for clust_num in clust_nums:
                clust = chrom.loc[chrom.cluster==clust_num]
                clust_hap = clust.iloc[0].cluster_hap
                clust_hap_imputed = clust.iloc[0].cluster_hap_imputed
                
                new_data_cluster = new_data.loc[(new_data.cell_index==ci)&(new_data.chr==chr_num)&
                                               (new_data.cluster==clust_num)]
                new_data.loc[new_data_cluster.index,'cluster_hap']=clust_hap
                new_data.loc[new_data_cluster.index,'cluster_hap_imputed']=clust_hap_imputed
    
    return new_data


#Populates manual parent of origin assignments for zygotes. Cluster assignment 1 corresponds to maternal origin,
# and cluster assignmennt 2 corresponds to paternal origin.
def assign_manual_zygote_calls(data):
    new_data = data.copy()
    zygotes = data.loc[data.stage=='zygote']
    new_data.loc[zygotes.index,'cluster_hap']=zygotes.cluster-1
    new_data.loc[zygotes.index,'cluster_hap_imputed']=zygotes.cluster-1
    
    return new_data

#Returns a dataframe with only euploid chromosomes
def get_euploid(data):
    data_c=data.copy()
    cell_indices=data.cell_index.unique()
    for cell_index in cell_indices:
        cell = data.loc[data.cell_index==cell_index]
        chr_nums=cell.chr.unique()
        for chr_num in chr_nums:
            chrom=cell.loc[cell.chr==chr_num]
            num_clusters = len(chrom.cluster.unique())
            if ((chr_num==20)&(num_clusters==1)):
                continue
            if num_clusters!=2:
                data_c=data_c.drop(index=chrom.index)
    return data_c

In [83]:
#get chromosome data and remove outliers and chromosomes with overlapping clusters
chromosomes = source_data.loc[(source_data.chr<21)&(source_data.inlier==1)&(source_data.cluster!=-1)]

#remove aneuploid chromosomes
euploid = get_euploid(chromosomes)

#get raw parent of origin estimates for each cluster, without using information from other clusters. 
raw_haps = call_raw_haps(euploid)

#Revise parent of origin assignments using info from other clusters. 
revised_haps = revise_raw_hap_calls(raw_haps)

#Transfer the parent of origin calls to the full data, such that outliers also get assigned parent of origin
#based on the cluster they are associated with.
annotated_data = assign_haps_to_outliers(source_data,revised_haps)

#Assign manual zygote parent of origin assignments to the cluster_hap and cluster_hap_imputed columns
annotated_data = assign_manual_zygote_calls(annotated_data)

In [ ]:
#write new annotations to file
annotated_data.to_csv("../data/Table_S1_pgp1_data_table.csv",index=False)